In [1]:
#Add needed imports
import numpy as np
import pandas as pd
import os
import shap
from numpy import mean
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras 
import pydot
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from keras.utils import plot_model
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import os
import shap

from tensorflow.compat.v1.keras.backend import get_session
tf.compat.v1.disable_v2_behavior()
#Read data
proccessed_data_path =os.path.join(os.path.pardir,os.path.pardir,'data','processed')
train_path = os.path.join(proccessed_data_path,'dataset10.csv')
df = pd.read_csv(train_path)
labels=df['Churn']
x = df.drop(columns=['Churn','Unnamed: 0'],axis = 'columns')
y=np.ravel(labels)
oversample = SMOTENC(categorical_features=[2])
x, y = oversample.fit_resample(x, y)
sc = RobustScaler()
x = pd.DataFrame(sc.fit_transform(x),columns = x.columns)
ii = 1

xgb_model = XGBClassifier(random_state=1,learning_rate=0.05, max_depth=7,eval_metric='mlogloss',use_label_encoder =False ,objective="binary:logistic")
dt_model=DecisionTreeClassifier(random_state=1,criterion='entropy',max_depth = 7,min_samples_leaf=30)  
nn_model = Sequential()
nn_model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.001), input_dim=5, activation='relu' ))
nn_model.add(Dropout(rate=0.2))
nn_model.add(Dense(8,kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu'))
nn_model.add(Dropout(rate=0.1))
nn_model.add(Dense(1, activation='sigmoid'))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1                             

Instructions for updating:
non-resource variables are not supported in the long term
Iteration 1


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.176695
                      0.481542
                      0.093554
                      0.561953
                      3.025895
 DT - feature_importance_vals
                     0.015542
                     0.017255
                     0.018963
                     0.070112
                     0.311308
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


keras is no longer supported, please use tf.keras instead.


 NN - feature_importance_vals
                     0.036059
                     0.016186
                     0.021167
                     0.066522
                     0.254797
Iteration 2
 XGB - feature_importance_vals
                      0.236461
                      0.393046
                      0.127892
                      0.832490
                      2.838597
 DT - feature_importance_vals
                     0.025015
                     0.012944
                     0.010940
                     0.086070
                     0.323735


...

 XGB - feature_importance_vals
                      0.256523
                      0.441880
                      0.106058
                      0.741223
                      2.613120
 DT - feature_importance_vals
                     0.015226
                     0.011058
                     0.017001
                     0.070953
                     0.365068
 NN - feature_importance_vals
                     0.062235
                     0.045789
                     0.065275
                     0.096802
                     0.515527
Iteration 9
 XGB - feature_importance_vals
                      0.254517
                      0.444805
                      0.132538
                      0.750862
                      2.603486
 DT - feature_importance_vals
                     0.018606
                     0.011777
                     0.014597
                     0.064246
                     0.382838


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 NN - feature_importance_vals
                     0.061589
                     0.036579
                     0.050149
                     0.106739
                     0.607112
Iteration 10


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.270703
                      0.449296
                      0.118051
                      0.759493
                      2.610472
 DT - feature_importance_vals
                     0.019647
                     0.017133
                     0.009721
                     0.067553
                     0.389836
 NN - feature_importance_vals
                     0.094557
                     0.053412
                     0.065467
                     0.153888
                     0.492469
